In [ ]:
%reload_kedro

In [ ]:
!pip install shutup

In [ ]:
import pandas as pd
from ydata_profiling import ProfileReport
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import shutup
shutup.please()

In [ ]:
dict_df = {
  "2023":catalog.load("grades_2023"),
  # "2022":catalog.load("grades_2022"),
  "2021":catalog.load("grades_2021"),
  # "":catalog.load("grades_2020"),
  "2019":catalog.load("grades_2019"),
  "2018":catalog.load("grades_2018")
}

for k, df in dict_df.items():
  df["year"] = k

df_original = pd.concat(dict_df.values()).reset_index(drop=True)
df_original

In [ ]:
def clean_string_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    Convert all string values in a pandas DataFrame to lowercase, strip leading/trailing spaces, 
    and replace spaces with underscores. Ignores non-string values.

    Parameters:
        df (pd.DataFrame): The input DataFrame to clean.

    Returns:
        pd.DataFrame: The modified DataFrame with cleaned string columns.
    """
    def clean_value(value):
        if isinstance(value, str):
            return value.lower().strip().replace(" ", "_")
        return value

    return df.applymap(clean_value)

In [ ]:
grade_map = {
    'primero': 1,
    'segundo': 2,
    'tercero': 3,
    'cuarto': 4,
    'quinto': 5,
    'sexto': 6,
    'séptimo': 7,
    'octavo': 8,
    'noveno': 9,
    'décimo': 10,
    'undécimo': 11
}

nra_map = {
  'bajo': 0,
  'medio': 1,
  'alto': 2
}

cols = df_original.columns.tolist()
cols = [x.lower().strip().replace(" ","_") for x in cols]
df_final = df_original.copy()
df_final.columns = cols
df_final = clean_string_columns(df_final)
df_final["cantera"] = df_final["cantera"].replace("académica", "academica")
df_final["nra_encoded"] = df_final['nra'].map(nra_map)
df_final["grade_encoded"] = df_final['grado'].map(grade_map)
df_final["cantera_encoded"] = 0
df_final.loc[df_final["cantera"]=="academica", "cantera_encoded"]=1
df_final

In [ ]:
""" Target """

print("Distribucion del",df_final["nra"].value_counts())

df_final["nra"].hist()



In [ ]:
# profile = ProfileReport(df_final, title="Profiling Report")
# profile.to_file("../data/08_reports/eda.html")

In [ ]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

courses = ['matemática', 'comunicación', 'inglés', 'ciencias', 'dpsc', 'educación_física', 'arte', 'conducta']

scaler = StandardScaler()
df_scaled = scaler.fit_transform(df_final[courses])
sse = {}
for k in range(1, 11):  # Probar con diferentes números de clusters, por ejemplo, de 1 a 10
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(df_scaled)
    sse[k] = kmeans.inertia_  # Suma de errores cuadráticos

# Graficar el método del codo
plt.figure(figsize=(10, 6))
plt.plot(list(sse.keys()), list(sse.values()), 'bx-')
plt.xlabel('Número de clusters')
plt.ylabel('SSE')
plt.title('Método del Codo')
plt.show()

optimal_k = 4 # Esto es solo un ejemplo, debes elegir el número después de ver el gráfico

# Ejecutar KMeans con el número óptimo de clusters
kmeans = KMeans(n_clusters=optimal_k, random_state=42)
df_final['cluster'] = kmeans.fit_predict(df_scaled)



In [ ]:
fig = px.parallel_coordinates(df_final[courses+["promedio","nra_encoded", "cluster"]], color="promedio")
fig.show()

In [ ]:
_df = round(df_final[courses+["promedio","nra_encoded", "cluster"]].corr()*100)
fig = px.imshow(_df,
                text_auto=True,  # Muestra los valores de correlación
                labels=dict(x="Variables", y="Variables", color="Correlación"),
                x=_df.columns,
                y=_df.columns,
                color_continuous_scale='RdBu_r',  # Escala de color para la correlación
                )

# Ajustes visuales adicionales
fig.update_layout(title='Mapa de Calor de la Matriz de Correlación',
                  width=1000, height=1000)

# Mostrar el gráfico
fig.show()

In [ ]:
""" Variable cantera con distribucion dispareja """
df_final["cantera"].value_counts()

In [ ]:
""" Variable sede no utilizable hasta corregir la data """
df_final["grado"].value_counts()

In [ ]:
for cantera in df_final["cantera"].unique():
  sub_df = df_final[df_final["cantera"] == cantera]
  axes = sub_df.hist(grid=False, figsize=(10, 8), bins=20, color='skyblue')
  plt.suptitle(f'Cantera {cantera} - Alumnos: {sub_df.shape[0]}')  # Añadir un título general a toda la figura
  plt.show()
